# Align
---

#### Overview
Interactive 3D alignment of serial sections.

In [1]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import renderapi

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import uploader
from interactive_render import plotting

#### `render-ws` environment variables

In [9]:
# create an authorized session
auth = basic_auth.load_auth()

# render-ws environment variables
# params_render = {
#     "host": "http://localhost",
#     "port": 8081,
#     "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
#     "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
#     "owner": "akievits",
#     "project": "20230914_RP_exocrine_partial_test",
#     "session": sesh
# }

params_render = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "auth": auth
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/")

## 2) Rough alignment (I)
---
Perform rough alignment of downsamples section images

### Create downsampled montage stack

In [3]:
from tifffile import TiffWriter
from dsstack import create_downsampled_stack

In [4]:
# Set stacks for downsampling
stack_2_downsample = {
    'in': 'postcorrection_stitched',
    'out': 'postcorrection_dsmontages'
}

In [10]:
# Create downsampled stack
ds_stack = create_downsampled_stack(dir_project, 
                                    stack_2_downsample,
                                    width=2048, 
                                    **params_render)
# Upload
# initialize uploader
uppity = uploader.Uploader(
        **params_render,
        clobber=False
)

# import stack to render-ws
uppity.upload_to_render(
    stacks=ds_stack,
    z_resolution=100
)

  0%|          | 0/3 [00:00<?, ?it/s]

TypeError: __init__() got an unexpected keyword argument 'session'